In [2]:
import torch

In [3]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [5]:
import os
import glob
import numpy as np
from PIL import Image 
import torch
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms

In [6]:
class celeba(Dataset):
    def __init__(self, data_path=None, label_path=None):
        self.data_path = data_path
        self.label_path = label_path

    
        self.transform = transforms.Compose(
            [transforms.Resize(224),
             transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    def __len__(self):
        return len(self.data_path)

    def __getitem__(self, idx):
        image_set = Image.open(self.data_path[idx])
        image_tensor = self.transform(image_set)
        image_label = torch.Tensor(self.label_path[idx])
        
        image_tensor = image_tensor.to('cuda')
        image_label = image_label.to('cuda')
        
        return image_tensor, image_label


In [7]:
def main():
    
    
    file_name = "/kaggle/input/celeba-val/img_align_celeba/"

    data_path = sorted(glob.glob(file_name + '*.jpg'))
    #print(len(data_path))

    label_path = "/kaggle/input/attributes/list_attr_celeba.txt"
    label_list = open(label_path).readlines()[2:]
    data_label = []
    for i in range(len(label_list)):
        data_label.append(label_list[i].split())

    for m in range(len(data_label)):
        data_label[m] = [n.replace('-1', '0') for n in data_label[m]][1:]
        data_label[m] = [int(p) for p in data_label[m]]

    #print(data_label)
    
    attributes = open(label_path).readlines()[1].split()
    #print(attributes)
    
    dataset = celeba(data_path, data_label)

    
    # split data into train, valid, test set as per list_eval_partition CelebA
    indices = list(range(202599))
    split_train = 162770
    split_valid = 182637
    train_idx, valid_idx, test_idx = indices[:split_train], indices[split_train:split_valid], indices[split_valid:]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, sampler=train_sampler)

    validloader = torch.utils.data.DataLoader(dataset, batch_size=64, sampler=valid_sampler)

    testloader =  torch.utils.data.DataLoader(dataset, batch_size=64, sampler=test_sampler)

    """
    print(len(trainloader))
    print(len(validloader))
    print(len(testloader))
    """
    
    return trainloader, validloader, testloader


In [7]:
train_dataloader = main()[0]
val_dataloader = main()[1]

In [8]:
print(len(train_dataloader))
print(len(val_dataloader))

2544
311


In [9]:
print(len(train_dataloader)*64)
print(len(val_dataloader)*64)

162816
19904


In [10]:
print(len(train_dataloader)*64*40)
print(len(val_dataloader)*64*40)

6512640
796160


In [8]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torchvision
#from dataloader import main

In [9]:
model = torchvision.models.resnet18(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 146MB/s] 


In [10]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [15]:
print("Og:", count_parameters(model))

Og: 11689512


In [16]:
for params in model.parameters():
    params.requires_grad = False

In [17]:
print("Freeze all:", count_parameters(model))

Freeze all: 0


In [11]:
in_features = model.fc.in_features
out_features = 40

class SigmoidClassifier(nn.Module):
    def __init__(self, in_features, out_features):
        super(SigmoidClassifier, self).__init__()
        self.fc = nn.Linear(in_features, out_features)

    def forward(self, x):
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

model.fc = SigmoidClassifier(in_features=in_features, out_features=out_features)
print(model)

'''
for params in model.fc.parameters():
    params.requires_grad = True

print("Unfreeze Last Layer:", count_parameters(model))
'''



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

'\nfor params in model.fc.parameters():\n    params.requires_grad = True\n\nprint("Unfreeze Last Layer:", count_parameters(model))\n'

In [12]:
model.to('cuda')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
type(model)

torchvision.models.resnet.ResNet

In [14]:
torch.cuda.is_available()

True

In [15]:
torch.cuda.device_count()

1

In [16]:
torch.cuda.current_device()

0

In [17]:
torch.cuda.get_device_name(0)


'Tesla P100-PCIE-16GB'

In [18]:
next(model.parameters()).device

device(type='cuda', index=0)

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

In [40]:
len(train_dataloader)

2544

In [42]:
1322/2544

0.5196540880503144

In [47]:
len(val_dataloader)

311

In [27]:
for epoch in range(20):
    model.train()
    print("Training")
    running_loss = 0
    correct_train = 0
    total_train = 0
    
    for data in train_dataloader:
        img, labels = data
        
        img = img.to('cuda')
        labels = labels.to('cuda')
        
        optimizer.zero_grad()
        
        pred = model(img)
        #pred = pred.to('cpu')
        
        loss = criterion(pred, labels)
        loss.backward()
        
        optimizer.step()
        
        res = pred > 0.5
        correct_train += (res == labels).sum().item()
        #total_train += labels.size(0)
        
        running_loss += loss.item()
        
    train_loss = running_loss/(len(train_dataloader)*64)
    train_acc = (correct_train/(len(train_dataloader)*40*64))
    
    model.eval()
    print("Validating")
    with torch.no_grad():
        running_loss_val = 0
        correct_val = 0
        total_val = 0
        
        for val_data in val_dataloader:
            val_img, val_labels = val_data
            
            val_img = val_img.to('cuda')
            val_lables = val_labels.to('cuda')
            
            val_pred = model(val_img)
            loss_val = criterion(val_pred, val_labels)
            running_loss_val += loss_val.item()
            
            val_res = val_pred > 0.5
            correct_val += (val_res == val_labels).sum().item()
            #total_val += val_labels.size(0)
    
    val_acc = correct_val/(len(val_dataloader)*40*64)
    val_loss = running_loss_val/(len(val_dataloader)*64)
            
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc*100:.2f}%, Val Accuracy: {val_acc*100:.2f}%, Val Loss: {val_loss:.4f}")
    
    
        
    #running_loss += loss
    #print(running_loss)

Training
Validating
Epoch 1, Train Loss: 0.4721, Train Accuracy: 80.22%, Val Accuracy: 80.89%, Val Loss: 0.4609
Training
Validating
Epoch 2, Train Loss: 0.4693, Train Accuracy: 81.05%, Val Accuracy: 81.05%, Val Loss: 0.4603
Training
Validating
Epoch 3, Train Loss: 0.4689, Train Accuracy: 81.17%, Val Accuracy: 81.02%, Val Loss: 0.4600
Training
Validating
Epoch 4, Train Loss: 0.4686, Train Accuracy: 81.16%, Val Accuracy: 81.26%, Val Loss: 0.4599
Training
Validating
Epoch 5, Train Loss: 0.4684, Train Accuracy: 81.17%, Val Accuracy: 80.83%, Val Loss: 0.4601
Training
Validating
Epoch 6, Train Loss: 0.4683, Train Accuracy: 81.21%, Val Accuracy: 80.90%, Val Loss: 0.4600
Training
Validating
Epoch 7, Train Loss: 0.4682, Train Accuracy: 81.23%, Val Accuracy: 80.62%, Val Loss: 0.4601
Training
Validating
Epoch 8, Train Loss: 0.4682, Train Accuracy: 81.26%, Val Accuracy: 80.88%, Val Loss: 0.4597
Training
Validating
Epoch 9, Train Loss: 0.4682, Train Accuracy: 81.25%, Val Accuracy: 80.67%, Val Loss:

In [28]:
checkpoint_path = '/kaggle/working/model_checkpoint.pth' 

In [29]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': val_loss,
    'accuracy': val_acc,
}, checkpoint_path)
